In [ ]:
from oauth2client.service_account import ServiceAccountCredentials
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from datetime import datetime
from gspread_formatting import *
from selenium import webdriver
from time import sleep
import numpy as np
import gspread
import time
import string
import gspread
import time
import string
import warnings

warnings.filterwarnings("ignore")


from selenium import webdriver
from selenium.webdriver.chrome.options import Options

driver = webdriver.Chrome(ChromeDriverManager().install())
driver.maximize_window()
credentials ={
  "type": "service_account",
  "project_id": "vital-pillar-329514",
  "private_key_id": "your private id",
  "private_key": "your private key",
  "client_email": "otcmarkets@vital-pillar-329514.iam.gserviceaccount.com",
  "client_id": "116521596231716126855",
  "auth_uri": "https://accounts.google.com/o/oauth2/auth",
  "token_uri": "https://oauth2.googleapis.com/token",
  "auth_provider_x509_cert_url": "https://www.googleapis.com/oauth2/v1/certs",
  "client_x509_cert_url": "https://www.googleapis.com/robot/v1/metadata/x509/otcmarkets%40vital-pillar-329514.iam.gserviceaccount.com"
}
css_ele=driver.find_element_by_css_selector
css_eles=driver.find_elements_by_css_selector
x_ele=driver.find_element_by_xpath
try:
    driver.maximize_window()
except:
       pass
values_available = False

headers = { 'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_2) AppleWebKit/601.3.9 (KHTML, like Gecko) Version/9.0.2 Safari/601.3.9'}



def styleDate(col_a, col_no, comparer):
    x_c = col_a + str(col_no)
    x_val = worksheet.acell(x_c).value
    if x_val == "Not available" and comparer != "Not available":
        fmt = cellFormat(backgroundColor=color(0, 0.9, 0.1), )
        format_cell_range(worksheet, x_c, fmt)
        print("updated value")
    if "/" in x_val and "/" in comparer:
        print("YESSSSSS")
        s = comparer.split("/")
        new = datetime(int(s[2]), int(s[0]), int(s[1]))
        today = datetime.now()
        today = today.date()
        new = new.date()
        print(today,new)
        if today == new:
            print("Format")
            fmt = cellFormat(backgroundColor=color(0, 0, 1), )
            format_cell_range(worksheet, x_c, fmt)
    elif comparer == "Not available":
        pass

def convert_date(d):
    s = d.split("/")
    new = datetime(int(s[2]), int(s[0]), int(s[1]))
    new = new.date()
    return new


# Style
def style(col_a, col_no, comparer):
    x_c = col_a + str(col_no)
    x_val = worksheet.acell(x_c).value
    if x_val == "Not available" and comparer != "Not available":
        fmt = cellFormat(backgroundColor=color(0, 0.9, 0.1), )
        format_cell_range(worksheet, x_c, fmt)
        print("updated value")
    elif x_val == "Not available":
        pass
    else:
        if type(comparer) == float:
            x_val = float(x_val)
            if comparer > x_val:
                fmt = cellFormat(backgroundColor=color(1, 0, 0), )
                format_cell_range(worksheet, x_c, fmt)
            if comparer < x_val:
                fmt = cellFormat(backgroundColor=color(0, 0.9, 0.1), )
                format_cell_range(worksheet, x_c, fmt)


# turns strings into float
def ft(x):
    x = float(x.replace(",", ""))
    print(x)
    return x


# turns floats into 2 decimal point
def td(x):
    x = float("{:.2f}".format(x))
    print(x)
    return x


def checkdate(x, i):
    if x == "Not Available":
        d = "Not available"
        return d
    else:
        d = css_eles('div[class="sc-bdVaJa fKInnm"]')[i].text.strip()
        return d


# Check if the value is available if it is then change it to float
def check(x, y, i):
    if x == "Not Available":
        x = "Not available"
        y = False
        d = "Not available"
        return x, y, d
    else:
        print(x)
        x = ft(x)
        y = True
        d = css_eles('div[class="sc-bdVaJa fKInnm"]')[i].text.strip()
        return x, y, d


'''
Takes two arguments main value and checks if values are available returns x_to, x_2 and changes availabilty value
'''


def math(availability, x):
    if availability:
        x_to = x * PSS
        x2 = x_to / AVD
        print("error", type(x2))
        x2 = td(x2)
        availability = False
        return x_to, x2, availability
    # If availabilty is false return Not available
    else:
        x_to = "Not available"
        x2 = "Not available"
        return x_to, x2, availability


sheet_name = "otcmarkets Scraper"
client = gspread.service_account_from_dict(credentials)
title = "Sheet2"
sh = client.open("otcmarkets Scraper")
worksheet = sh.worksheets()[1]
s = worksheet.col_values(1)[1:]
css_ele = driver.find_element_by_css_selector
css_eles = driver.find_elements_by_css_selector

Data_temp = []
# Start count is 2 because first row and col contains header
row_count = 2
col_no = 2
try : 
  driver.get("https://www.otcmarkets.com/market-activity/current-market")
  sleep(5)
  securities = css_eles('div.kYmYWE div:nth-child(2)')[0].text.strip()
  securities = ft(securities)
  dollar_volume = driver.find_element_by_css_selector('div[class="JpE3efVpgX"] p').text.strip()
  dollar_volume = ft(dollar_volume)
  AVD = dollar_volume / securities

except:
  pass
for currency in s:
    driver.get(f"https://www.otcmarkets.com/stock/{currency}/security")
    sleep(5)
    # Using PSS and ADV early cause we'll be neeeding its values in math()
    market_cap = css_eles('div[class="_335BJ2NvZW"] div[class^="sc-bdVaJa k"]')[0].text.strip()
    Authorized_Shares = css_eles('div[class="_335BJ2NvZW"] div[class^="sc-bdVaJa k"]')[1].text.strip()
    outstanding_Shares = css_eles('div[class="_335BJ2NvZW"] div[class^="sc-bdVaJa k"]')[2].text.strip()
    restricted_Shares = css_eles('div[class="_335BJ2NvZW"] div[class^="sc-bdVaJa k"]')[3].text.strip()
    unrestricted_Shares = css_eles('div[class="_335BJ2NvZW"] div[class^="sc-bdVaJa k"]')[4].text.strip()
    Held_at_DTC = css_eles('div[class="_335BJ2NvZW"] div[class^="sc-bdVaJa k"]')[5].text.strip()
    Par_Value = css_eles('div[class="_335BJ2NvZW"] div[class^="sc-bdVaJa k"]')[7].text.strip()
    Float = css_eles('div[class="_335BJ2NvZW"] div[class^="sc-bdVaJa k"]')[6].text.strip()
    try:
        PSS = driver.find_elements_by_tag_name('h2')[0].text
        PSS = ft(PSS)
        O = check(outstanding_Shares, values_available, 2)
        outstanding_Shares = O[0]
        values_available = O[1]
        oustanding_date = O[2]
        O = math(values_available, outstanding_Shares)
        outstanding_to = O[0]
        outstanding_2 = O[1]
        values_available = O[2]

        # Restricted share
        R = check(restricted_Shares, values_available, 3)
        restricted_Shares = R[0]
        values_available = R[1]
        restricted_date = R[2]
        R = math(values_available, restricted_Shares)
        restricted_to = R[0]
        restricted_2 = R[1]
        values_available = R[2]
        # Unristricted
        # Using indexes to fecth the 1st and 2nd return valyes
        U = check(unrestricted_Shares, values_available, 4)
        unrestricted_Shares = U[0]
        values_available = U[1]
        unrestricted_date = U[2]
        U = math(values_available, unrestricted_Shares)
        unrestricted_to = U[0]
        unrestricted_2 = U[1]
        values_available = U[2]
        # Float
        F = check(Float, values_available, 6)
        Float = F[0]
        values_available = F[1]
        float_date = F[2]
        F = math(values_available, Float)
        float_to = F[0]
        float_2 = F[1]
        values_available = F[2]
        if outstanding_Shares == "Not available" or unrestricted_Shares == "Not available":
            l_col = "Not available"
        else:
            l_col = outstanding_Shares - unrestricted_Shares
            l_col = l_col * PSS
            l_col = l_col / AVD
        if outstanding_Shares == "Not available" or restricted_Shares == "Not available":
            M_col = "Not available"
        else:
            M_col = outstanding_Shares - restricted_Shares
            M_col = M_col / AVD

    except:
        outstanding_to = "Not available"
        outstanding_2 = "Not available"
        restricted_to = "Not available"
        restricted_2 = "Not available"
        unrestricted_to = "Not available"
        unrestricted_2 = "Not available"
        float_to = "Not available"
        float_2 = "Not available"
        l_col = "Not available"

        oustanding_date = checkdate(outstanding_Shares, 2)
        restricted_date = checkdate(restricted_Shares, 3)
        unrestricted_date = checkdate(unrestricted_Shares, 4)
        float_date = checkdate(Float, 6)

    try:
        transfer_agent = css_ele('div[class="sc-bdVaJa gSGJZD"] a').text.strip()
    except:
        transfer_agent = ''
    try:
        transfer_agent_V = css_ele('a[href="/glossary#transfer-agent-verified"]')
        transfer_agent_V = "Transfer Agent Verified"
    except:
        transfer_agent_V = "Not Verified"


    c_date = datetime.now().strftime("%d-%m-%Y")
    stock = "bitcoin"
    array = np.array(
        [[currency, c_date, float_to, float_2, float_date, unrestricted_to, unrestricted_2, unrestricted_date,
          outstanding_to, outstanding_2, oustanding_date, l_col,M_col]])
    print(array)
    r = "A" + str(row_count)
    # formatting start
    style("C", row_count, float_to)
    style("D", row_count, float_2)
    styleDate("E", row_count, float_date)
    style("F", row_count, unrestricted_to)
    style("G", row_count, unrestricted_2)
    styleDate("H", row_count, unrestricted_date)
    style("I", row_count, outstanding_to)
    style("J", row_count, outstanding_2)
    styleDate("K", row_count, oustanding_date)
    style("L", row_count, l_col)
    style("M",row_count,M_col)
    # Formatting end

    worksheet.update(r, array.tolist())
    # New record at every next row
    row_count = row_count + 1
    # to check value at next colums
    col_no = col_no + 1
    sleep(10)
driver.close()
print('---------------Scraping & Updation Finished----------------------------------')
